# PyConAr 2018 - Taller API Georef

Para leer la guía de uso de la API, hacer click [acá](https://apis.datos.gob.ar/georef).

Para leer la referencia completa de la API, hacer click [acá](https://datosgobar.github.io/georef-ar-api/open-api/).

Requerimientos del taller:
 - Python 3
 - jupyter
 - pandas
 - requests

In [1]:
import os
import sys
import json
import pandas as pd
import requests
assert sys.version_info[0] == 3, 'El taller requiere Python 3'

## Capas de datos utilizadas por la API

La API utiliza varias capas de datos publicados por INDEC e IGN como fuentes de datos principales. A continuación, se muestra una representación visual de cada capa de datos utilizada:

### Provincias
Jurisdicciones de primer orden: 23 provincias y la Ciudad Autónoma de Buenos Aires. Cubren todo el territorio argentino. Fuente: **IGN**.

![title](img/provincias.png)

### Departamentos
Divisiones territoriales de segundo orden: 529 departamentos/partidos/comunas. Cubren todo el territorio argentino. Fuente: **IGN**.

![title](img/departamentos.png)
    
### Municipios
Entidades jurídicas con gobierno propio, definidas por los gobiernos provinciales. **No** cubren todo el territorio argentino. Fuente: **IGN**.

![title](img/municipios.png)

### Localidades
Asentamientos humanos (localidad censal o paraje, sitio edificado) según ([BAHRA](http://www.bahra.gob.ar/)). A diferencia de las provincias, departamentos y municipios, las localidades **no** tienen área, son datos en forma de puntos. Fuentes: **INDEC/IGN/Ministerio de Educación**.

![title](img/localidades.png)

### Calles
Listado de vías de circulación. Fuente: **INDEC**.

![title](img/calles.png)

Detalle de **Córdoba Capital**:

![title](img/calles_cba.png)

## Recursos (*endpoints*) de la API
- `/provincias`
- `/departamentos`
- `/municipios`
- `/localidades`
- `/direcciones`
- `/calles`
- `/ubicacion`

## Ejemplos rápidos

A continuación, se muestran algunos ejemplos rápidos de uso de la API, utilizando `requests`. Primero, se define el método `georef_response`, el cual imprime respuestas de la API en formato JSON indentado.

In [2]:
def georef_response(url):
    # Realizar una consulta GET a la API e imprimir la respuesta
    resp = requests.get('https://apis.datos.gob.ar/georef/api{}'.format(url))
    if resp.headers['Content-Type'] == 'application/json':
        return print(json.dumps(resp.json(), indent=4, ensure_ascii=False, sort_keys=True))
    else:
        print(resp.text)

### Provincias (`/provincias`)

In [3]:
georef_response('/provincias?nombre=pampa')

{
    "cantidad": 1,
    "inicio": 0,
    "provincias": [
        {
            "centroide": {
                "lat": -37.131554,
                "lon": -65.446655
            },
            "id": "42",
            "nombre": "La Pampa"
        }
    ],
    "total": 1
}


Otros parámetros a probar: `formato`, `id`, `interseccion`, `orden`, `aplanar`, `campos`, `max`, `inicio`, `exacto`.

### Departamentos (`/departamentos`)

In [4]:
georef_response('/departamentos?nombre=belgrano&max=1')

{
    "cantidad": 1,
    "departamentos": [
        {
            "centroide": {
                "lat": -32.735612,
                "lon": -66.715007
            },
            "id": "74014",
            "nombre": "Belgrano",
            "provincia": {
                "id": "74",
                "nombre": "San Luis"
            }
        }
    ],
    "inicio": 0,
    "total": 8
}


Otros parámetros a probar: `formato`, `id`, `interseccion`, `provincia`, `orden`, `aplanar`, `campos`, `inicio`, `exacto`.

### Municipios (`/municipios`)

In [5]:
georef_response('/municipios?provincia=14&max=1')

{
    "cantidad": 1,
    "inicio": 0,
    "municipios": [
        {
            "centroide": {
                "lat": -34.349352,
                "lon": -63.436578
            },
            "id": "140637",
            "nombre": "Melo",
            "provincia": {
                "id": "14",
                "nombre": "Córdoba"
            }
        }
    ],
    "total": 426
}


Otros parámetros a probar: `formato`, `id`, `interseccion`, `nombre`, `orden`, `aplanar`, `campos`, `inicio`, `exacto`.


### Localidades (`/localidades`)

In [6]:
georef_response('/localidades?provincia=tdf&max=1')

{
    "cantidad": 1,
    "inicio": 0,
    "localidades": [
        {
            "centroide": {
                "lat": -54.637686,
                "lon": -67.766941
            },
            "departamento": {
                "id": "94015",
                "nombre": "Ushuaia"
            },
            "id": "94015010000",
            "municipio": {
                "id": null,
                "nombre": null
            },
            "nombre": "LAGUNA ESCONDIDA",
            "provincia": {
                "id": "94",
                "nombre": "Tierra del Fuego, Antártida e Islas del Atlántico Sur"
            },
            "tipo": "Localidad simple (LS)"
        }
    ],
    "total": 4
}


Otros parámetros a probar: `formato`, `id`, `nombre`, `departamento`, `municipio`, `orden`, `aplanar`, `campos`, `inicio`, `exacto`.


### Direcciones (`/direcciones`)

In [7]:
georef_response('/direcciones?direccion=santa fe 1000&departamento=quilmes&max=1')

{
    "cantidad": 1,
    "direcciones": [
        {
            "altura": 1000,
            "departamento": {
                "id": "06658",
                "nombre": "Quilmes"
            },
            "id": "0665801004045",
            "nombre": "SANTA FE",
            "nomenclatura": "SANTA FE 1000, Quilmes, Buenos Aires",
            "provincia": {
                "id": "06",
                "nombre": "Buenos Aires"
            },
            "tipo": "CALLE",
            "ubicacion": {
                "lat": null,
                "lon": null
            }
        }
    ],
    "inicio": 0,
    "total": 2
}


Otros parámetros a probar: `formato`, `id`, `provincia`, `orden`, `aplanar`, `campos`, `inicio`, `exacto`, `tipo`.


### Calles (`/calles`)

In [8]:
georef_response('/calles?provincia=neuquen&max=1')

{
    "calles": [
        {
            "altura": {
                "fin": {
                    "derecha": 0,
                    "izquierda": 0
                },
                "inicio": {
                    "derecha": 0,
                    "izquierda": 0
                }
            },
            "departamento": {
                "id": "58112",
                "nombre": "Zapala"
            },
            "id": "5811204000335",
            "nombre": "CALLE S N",
            "nomenclatura": "CALLE S N, Zapala, Neuquén",
            "provincia": {
                "id": "58",
                "nombre": "Neuquén"
            },
            "tipo": "CALLE"
        }
    ],
    "cantidad": 1,
    "inicio": 0,
    "total": 2325
}


Otros parámetros a probar: `formato`, `id`, `nombre`, `departamento`, `orden`, `aplanar`, `campos`, `inicio`, `exacto`, `tipo`.

### Ubicación (`/ubicacion`)

In [9]:
georef_response('/ubicacion?lat=-27.2741&lon=-66.7529')

{
    "ubicacion": {
        "departamento": {
            "id": "10035",
            "nombre": "Belén"
        },
        "lat": -27.2741,
        "lon": -66.7529,
        "municipio": {
            "id": "100077",
            "nombre": "Hualfín"
        },
        "provincia": {
            "id": "10",
            "nombre": "Catamarca"
        }
    }
}


Otros parámetros a probar: `formato`, `aplanar`, `campos`.

## Lotes de datos
Para trabajar con archivos de datos CSV, es conveniente utilizar los recursos versión `POST` de la API, ya que los mismos permiten envíar varias consultas en una misma petición HTTP. Para lograr esto, se definen a continuación algunas funciones para trabajar con datos cargados en DataFrames.

### Funciones de utilidad

Se definen dos funciones de utilidad:

#### `load_csv`
La función `load_csv` carga un archivo CSV y toma una muestra aleatoria de 100 elementos (por defecto). El muestreado se realiza para poder agilizar el taller y no tener que manejar cantidades de datos demasiado grandes.

In [10]:
def load_csv(name, length=100, random_state=1):
    # Cargar archivo CSV de la carpeta 'datos', eliminando duplicados
    dataframe = pd.read_csv(os.path.join('datos', name)).drop_duplicates()
    
    # Tomar una muestra 'aleatoria' de N elementos
    dataframe = dataframe.sample(n=min(len(dataframe.index), length), random_state=random_state)
    
    # Asegurarse de que los índices sean secuenciales (0, 1, 2, 3...)
    return dataframe.reset_index(drop=True)

#### `expand_geojson_column`
La función `expand_geojson_column` toma un DataFrame con una columna `geojson`, y devuelve una copia del DataFrame recibido con dos columnas agregadas: `longitud` y `latitud`, con los valores correspondientes de la columna original. El contenido de cada celda de la columna `geojson` debe ser de tipo `Point`.

In [11]:
def expand_geojson_column(data):
    # Toma un DataFrame con una columna 'geojson' y retorna un DataFrame con 'lat' y 'lon' agregado
    def geojson_lat_lon(value):
        geojson = json.loads(value)
        return geojson['coordinates']
    
    col = data[['geojson']].applymap(geojson_lat_lon)
    return col['geojson'].apply(pd.Series).rename(columns={0: 'longitud', 1: 'latitud'}).join(data)

Como ejemplo, se construye un DataFrame `geojson_df`, y se lo une con el resultado de expandir su columna `geojson` a `longitud` y `latitud`:

In [12]:
geojson_df = pd.DataFrame([
    '{"type":"Point","coordinates":[-65.5497560941612,-22.3231238419683]}',
    '{"type":"Point","coordinates":[-66.1589398249571,-22.3505830769888]}'
], columns=['geojson'])
geojson_df

,geojson
0,"{""type"":""Point"",""coordinates"":[-65.54975609416..."
1,"{""type"":""Point"",""coordinates"":[-66.15893982495..."


In [13]:
expand_geojson_column(geojson_df)

,longitud,latitud,geojson
0,-65.549756,-22.323124,"{""type"":""Point"",""coordinates"":[-65.54975609416..."
1,-66.158940,-22.350583,"{""type"":""Point"",""coordinates"":[-66.15893982495..."


### Funciones de normalización

Antes definir las funciones de normalización, es necesario notar que las versiones `POST` de los recursos de la API toman los mismos parámetros que sus versiones `GET`, con la diferencia de que los valores se envían en el cuerpo de la petición HTTP en lugar del *query string*. Como ejemplo, las siguientes tres consultas:

```m
GET https://apis.datos.gob.ar/georef/api/provincias?nombre=cordoba&campos=nombre
GET https://apis.datos.gob.ar/georef/api/provincias?nombre=chaco
GET https://apis.datos.gob.ar/georef/api/provincias?nombre=san luis&aplanar
```

Son equivalentes a la siguiente:
```json
// POST https://apis.datos.gob.ar/georef/api/provincias
// Content-type: application/json
{
    "provincias": [
        {
            "nombre": "cordoba",
            "campos": "nombre"
        },
        {
            "nombre": "chaco"
        },
        {
            "nombre": "san luis",
            "aplanar": true
        }
    ]
}

```

Habiendo dicho esto, se pueden definir las dos funciones:

#### `georef_normalize_entities`
La función `georef_normalize_entities` toma un DataFrame e intenta normalizar sus datos utilizando la API. Para utilizar la función, es necesario especificar qué recurso (`entity`) se va a consultar. Los valores aceptados por este parámetro son: `provincias`, `departamentos`, `municipios`, `localidades`, `direcciones` y `calles`. Por el otro lado, el parámetro `field` debe contener un diccionario que especifica cuáles campos del DataFrame corresponden a cada parámetro propio de la API.

In [14]:
def georef_normalize_entities(data, entity, fields, params=None, prefix=''):
    if not params:
        params = {}
    queries = []

    for i in data.index:
        # Se crea cada 'consulta' individual (equivalente a hacer un request GET), y
        # se la agrega a una lista de consultas.
        query = {
            'max': 1,
            'aplanar': True,
            **params
        }

        for key, value in fields.items():
            query[key] = str(data.loc[i, value])

        queries.append(query)

    body = {
        entity: queries
    }

    url = 'https://apis.datos.gob.ar/georef/api/{}'.format(entity)
    
    # Se envía la lista de consultas utilizando la versión POST del recurso
    resp = requests.post(url, json=body)
    resp.raise_for_status()
    results = resp.json()['resultados']
    
    # Se crea un nuevo DataFrame utilizando el primer elemento del resultado de cada
    # consulta como filas
    normalized = pd.DataFrame(result[entity][0] if result[entity] else {} for result in results)
    return normalized.add_prefix(prefix).join(data)

A modo de ejemplo, se construye un DataFrame con un listado de provincias:

In [15]:
df_example = pd.DataFrame([
    'CABA',
    'Ciudad de Buenos Aires',
    'Capital federal',
    'Stgo del Estero',
    'Tierra del Fuego',
    'Tucuman'
], columns=['prov'])
df_example

,prov
0,CABA
1,Ciudad de Buenos Aires
2,Capital federal
3,Stgo del Estero
4,Tierra del Fuego
5,Tucuman


Luego, se normaliza el DataFrame utilizando `georef_normalize_entities`, con entidad `provincias`:

In [16]:
georef_normalize_entities(df_example, 'provincias', {'nombre': 'prov'})

,centroide_lat,centroide_lon,id,nombre,prov
0,-34.614493,-58.445856,02,Ciudad Autónoma de Buenos Aires,CABA
1,-34.614493,-58.445856,02,Ciudad Autónoma de Buenos Aires,Ciudad de Buenos Aires
2,-34.614493,-58.445856,02,Ciudad Autónoma de Buenos Aires,Capital federal
3,-27.782412,-63.252387,86,Santiago del Estero,Stgo del Estero
4,-82.521518,-50.742749,94,"Tierra del Fuego, Antártida e Islas del Atlánt...",Tierra del Fuego
5,-26.947800,-65.364758,90,Tucumán,Tucuman


#### `georef_reverse_geocode`
La función `georef_reverse_geocode` toma un DataFrame con coordenadas (latitud y longitud) y realiza una geocodificación inversa de los datos utilizando la API. El parámetro `fields` debe tomar el valor de un diccionario que indique cuáles columnas del DataFrame corresponden a los parámetros `lat` y `lon` de la API.

In [17]:
def georef_reverse_geocode(data, fields, params=None, prefix=''):
    if not params:
        params = {}
    queries = []

    for i in data.index:
        # Se crea cada 'consulta' individual (equivalente a hacer un request GET), y
        # se la agrega a una lista de consultas.
        query = {
            'aplanar': True,
            **params
        }

        for key, value in fields.items():
            query[key] = data.loc[i, value]

        queries.append(query)

    body = {
        'ubicaciones': queries
    }

    # Se envía la lista de consultas utilizando la versión POST del recurso /ubicacion
    resp = requests.post('https://apis.datos.gob.ar/georef/api/ubicacion', json=body)
    resp.raise_for_status()
    results = resp.json()['resultados']

    # Se crea un nuevo DataFrame con los resultados de cada consulta como filas
    geocoded =  pd.DataFrame(result['ubicacion'] for result in results).drop(columns=['lon', 'lat'])
    return geocoded.add_prefix(prefix).join(data)

A modo de ejemplo, se construye un DataFrame con un listado de coordenadas:

In [18]:
df_example2 = pd.DataFrame([
    (-66.222703465532,-22.0225667586346),
    (-66.3782703614848,-22.7015782843536),
    (-68.4427937929881,-29.005534118598)
], columns=['longitud', 'latitud'])
df_example2

,longitud,latitud
0,-66.222703,-22.022567
1,-66.378270,-22.701578
2,-68.442794,-29.005534


Luego, se realiza la geocodificación inversa de los datos con `georef_reverse_geocode`:

In [19]:
georef_reverse_geocode(df_example2, {'lat': 'latitud', 'lon': 'longitud'})

,departamento_id,departamento_nombre,municipio_id,municipio_nombre,provincia_id,provincia_nombre,longitud,latitud
0,38077,Santa Catalina,386147,Santa Catalina,38,Jujuy,-66.222703,-22.022567
1,38049,Rinconada,386070,Mina Pirquitas,38,Jujuy,-66.378270,-22.701578
2,46077,General Lamadrid,460077,General Lamadrid,46,La Rioja,-68.442794,-29.005534


### Ejemplos con recursos concretos

A continuación se cargan distintos recursos en formato CSV, y se los normaliza utilizando las funciones definidas anteriormente.

Primero, leemos los archivos CSV utilizando `load_csv`:

In [20]:
df_bioetanol = load_csv('bioetanol.csv')
df_justicia = load_csv('centros-acceso-justicia-activos.csv')
df_rencap = load_csv('denuncias-rencap-2018-10-02.csv')
df_minerales = load_csv('geologa-climatologa-yacimientos-minerales.csv')
df_puntos_digitales = load_csv('puntos-digitales.csv')
df_arsat_internet = load_csv('arsat-internet.csv')

#### Ventas de Bioetanol
Recurso:
- **Título:** Bioetanol: detalle de cupos asignados, producción y ventas, por empresa.
- **Descripción:** Datos mensuales de producción y ventas al corte de bioetanol en metros cúbicos, por empresa y provincia.
- **Fuente**: [datos.gob.ar](http://datos.gob.ar/dataset/energia-estadisticas-biodiesel-bioetanol)

In [21]:
df_bioetanol.head()

,fecha,empresa,localidad,provincia,insumo,cupo_total_asignado_m3,produccion__m3,ventas_totales__m3,ventas_a_axion_m3,ventas_a_petrobras_m3,ventas_a_fox_petrol_m3,ventas_a_refineria_del_norte_m3,ventas_a_shell_m3,ventas_a_ypf_m3,ventas_a_energia_derivada_del_petroleo_m3,ventas_a_oil_combustibles_m3,ventas_a_trafigura_oil_m3
0,8/1/2015,BIOTRINIDAD S.A.,VILLA LA TRINIDAD,TUCUMÁN,CAÑA DE AZÚCAR,1912.0,4420.176879,2001.718256,0.00,0.00,0.0,0.00,0.00,2001.72,0.0,0.00,0.0
1,11/1/2015,COMPAÑÍA BIONERGÉTICA LA FLORIDA S.A.,LA FLORIDA,TUCUMÁN,CAÑA DE AZÚCAR,7848.0,8532.444725,7937.511055,967.93,545.62,0.0,1762.11,1245.90,3268.57,0.0,147.38,0.0
2,9/1/2013,BIO SAN ISIDRO S.A.,CAMPO SANTO,SALTA,CAÑA DE AZÚCAR,1025.0,1002.097284,955.173721,0.00,0.00,0.0,0.00,0.00,955.17,0.0,0.00,0.0
3,9/1/2016,BIOLEDESMA S.A.,LEDESMA,JUJUY,CAÑA DE AZÚCAR,8556.0,8714.150347,8542.569931,770.57,0.00,0.0,0.00,3113.46,4241.66,0.0,416.88,0.0
4,12/1/2013,BIO SAN ISIDRO S.A.,CAMPO SANTO,SALTA,CAÑA DE AZÚCAR,1061.0,1103.550221,1039.254580,0.00,0.00,0.0,0.00,0.00,1039.25,0.0,0.00,0.0


Se puede utilizar el recurso `/localidades` de la API para normalizar el DataFrame. De esta forma, podemos conseguir el ID oficial de la localidad y provincia para cada fila de datos. Se agrega el prefixo `gr_` a las columnas agregadas para mayor claridad.

In [22]:
georef_normalize_entities(
    df_bioetanol,
    'localidades',
    {'nombre': 'localidad', 'provincia': 'provincia'},
    {'campos': 'id, nombre, provincia.id'},
    prefix='gr_'
)

,gr_id,gr_nombre,gr_provincia_id,fecha,empresa,localidad,provincia,insumo,cupo_total_asignado_m3,produccion__m3,ventas_totales__m3,ventas_a_axion_m3,ventas_a_petrobras_m3,ventas_a_fox_petrol_m3,ventas_a_refineria_del_norte_m3,ventas_a_shell_m3,ventas_a_ypf_m3,ventas_a_energia_derivada_del_petroleo_m3,ventas_a_oil_combustibles_m3,ventas_a_trafigura_oil_m3
0,NaN,NaN,NaN,8/1/2015,BIOTRINIDAD S.A.,VILLA LA TRINIDAD,TUCUMÁN,CAÑA DE AZÚCAR,1912.0,4420.176879,2001.718256,0.000000,0.000000,0.00,0.00,0.000000,2001.720000,0.00,0.000000,0.00000
1,90014080002,LA FLORIDA,90,11/1/2015,COMPAÑÍA BIONERGÉTICA LA FLORIDA S.A.,LA FLORIDA,TUCUMÁN,CAÑA DE AZÚCAR,7848.0,8532.444725,7937.511055,967.930000,545.620000,0.00,1762.11,1245.900000,3268.570000,0.00,147.380000,0.00000
2,66049010000,CAMPO SANTO,66,9/1/2013,BIO SAN ISIDRO S.A.,CAMPO SANTO,SALTA,CAÑA DE AZÚCAR,1025.0,1002.097284,955.173721,0.000000,0.000000,0.00,0.00,0.000000,955.170000,0.00,0.000000,0.00000
3,38035080002,PUEBLO LEDESMA,38,9/1/2016,BIOLEDESMA S.A.,LEDESMA,JUJUY,CAÑA DE AZÚCAR,8556.0,8714.150347,8542.569931,770.570000,0.000000,0.00,0.00,3113.460000,4241.660000,0.00,416.880000,0.00000
4,66049010000,CAMPO SANTO,66,12/1/2013,BIO SAN ISIDRO S.A.,CAMPO SANTO,SALTA,CAÑA DE AZÚCAR,1061.0,1103.550221,1039.254580,0.000000,0.000000,0.00,0.00,0.000000,1039.250000,0.00,0.000000,0.00000
5,90056090000,VILLA DE LEALES,90,2/1/2018,BIOENERGÉTICA LEALES S.A.,LEALES,TUCUMÁN,CAÑA DE AZÚCAR,NaN,2180,3186.000000,0.000000,0.000000,0.00,0.00,0.000000,3186.000000,0.00,0.000000,0.00000
6,14056010000,ALEJANDRO ROCA,14,5/1/2014,PROMAÍZ S.A.,ALEJANDRO ROCA,CÓRDOBA,MAÍZ,11250.0,12553.9103,11736.184460,2196.660000,719.200000,37.93,0.00,1496.960000,7285.430000,0.00,0.000000,0.00000
7,66126030000,EL TABACAL,66,7/1/2014,ALCONOA S.R.L.,EL TABACAL,SALTA,CAÑA DE AZÚCAR,3374.0,551.004422,3426.986734,262.000000,0.000000,0.00,0.00,2469.000000,695.990000,0.00,0.000000,0.00000
8,90077010001,AGUILARES,90,4/1/2012,ENERGÍAS ECOLOGICAS DEL TUCUMÁN S.A.,AGUILARES,TUCUMÁN,CAÑA DE AZÚCAR,2090.0,1778.900821,1897.662666,1154.770000,0.000000,0.00,0.00,481.360000,0.000000,0.00,261.530000,0.00000
9,90077010001,AGUILARES,90,2/1/2013,ENERGÍAS ECOLOGICAS DEL TUCUMÁN S.A.,AGUILARES,TUCUMÁN,CAÑA DE AZÚCAR,2090.0,1776.373973,1950.726469,1144.660000,0.000000,0.00,0.00,510.420000,0.000000,0.00,295.640000,0.00000


#### Denuncias RENCAP
Recurso:
- **Título:** Denuncias a empresas y trabajadores de control de admisión y permanencia en RENCAP - actualizado al 2018-10-02.
- **Descripción:** Denuncias recibidas en el Registro de Empresas y Trabajadores de Control de Admisión y Permanencia - RENCAP, a través del 0800-222-0080 o de la página web institucional http://www.argentina.gob.ar/justicia/rencap ó http://complementos.jus.gov.ar/denuncias/.
- **Fuente**: [datos.gob.ar](http://datos.gob.ar/dataset/justicia-denuncias-empresas-trabajadores-control-admision-permanencia-rencap)

In [23]:
df_rencap.head()

,denuncia_numero,hecho_fecha_hora,hecho_provincia,hecho_municipio,hecho_localidad,hecho_discriminacion,hecho_irregularidades,hecho_lugar,hecho_cometido_por,hecho_tipo_violencia,hecho_accion_realizada,denuncia_estado,denuncia_fecha_cambio_estado,denuncia_via_ingreso,denuncia_fecha_hora_carga,hecho_provincia_indec_id
0,973,2016-03-24 02:00:00,Ciudad Autónoma de Buenos Aires,Ciudad de Buenos Aires,Palermo,Por aspecto físico - Racial,NaN,En el acceso local,Controladores,NaN,Se solicitó asistencia policial,Derivado,2016-04-08 19:00:29.607,Call Center,2016-04-07 17:28:19.923,2
1,544,2013-06-30 02:00:00,Ciudad Autónoma de Buenos Aires,Ciudad de Buenos Aires,Palermo,Por aspecto físico - Por indumentaria - Racial,El personal de control no esta identificado,En el acceso local,Controladores - Personal de empresas de seguri...,NaN,NaN,Derivado,2013-07-24 12:23:03.050,Página web de RENCAP,2013-07-01 23:27:32.323,2
2,553,2013-07-18 21:30:00,Ciudad Autónoma de Buenos Aires,Barrio Palermo,NaN,Por aspecto físico,NaN,En el acceso local,Controladores,Agresión verbal,NaN,Derivado,2013-07-24 12:45:04.487,Call Center,2013-07-19 12:48:28.177,2
3,273,2012-03-11 04:30:00,Buenos Aires,Florencio Varela,Fcio. Varela,NaN,NaN,En las proximidades del local,Controladores,Agresión verbal - Agresión física,Se hizo denuncia policial,Derivado,2012-03-14 12:56:23.353,Call Center,2012-03-13 17:12:25.580,6
4,1209,2017-08-12 01:00:00,Tucumán,San Miguel de Tucumán,SAN MIGUEL DE TUCUMAN,Racial,El personal de control no esta identificado - ...,En el acceso local,Personas vinculadas a la empresa,NaN,NaN,Derivado,2017-08-14 16:47:16.497,Call Center,2017-08-14 09:27:25.947,90


Si se deseara normalizar la columna `hecho_municipio` del DataFrame, se podría utilizar el recurso `/municipios` de la API. El resultado sería el ID y nombre oficiales del municipio y los de su provincia. Se agrega nuevamente el prefixo `gr_` a las columnas agregadas para mayor claridad.

In [24]:
georef_normalize_entities(
    df_rencap,
    'municipios',
    {'nombre': 'hecho_municipio', 'provincia': 'hecho_provincia'},
    {'campos': 'id, nombre, provincia.id, provincia.nombre'},
    prefix='gr_'
)

,gr_id,gr_nombre,gr_provincia_id,gr_provincia_nombre,denuncia_numero,hecho_fecha_hora,hecho_provincia,hecho_municipio,hecho_localidad,hecho_discriminacion,hecho_irregularidades,hecho_lugar,hecho_cometido_por,hecho_tipo_violencia,hecho_accion_realizada,denuncia_estado,denuncia_fecha_cambio_estado,denuncia_via_ingreso,denuncia_fecha_hora_carga,hecho_provincia_indec_id
0,NaN,NaN,NaN,NaN,973,2016-03-24 02:00:00,Ciudad Autónoma de Buenos Aires,Ciudad de Buenos Aires,Palermo,Por aspecto físico - Racial,NaN,En el acceso local,Controladores,NaN,Se solicitó asistencia policial,Derivado,2016-04-08 19:00:29.607,Call Center,2016-04-07 17:28:19.923,2
1,NaN,NaN,NaN,NaN,544,2013-06-30 02:00:00,Ciudad Autónoma de Buenos Aires,Ciudad de Buenos Aires,Palermo,Por aspecto físico - Por indumentaria - Racial,El personal de control no esta identificado,En el acceso local,Controladores - Personal de empresas de seguri...,NaN,NaN,Derivado,2013-07-24 12:23:03.050,Página web de RENCAP,2013-07-01 23:27:32.323,2
2,NaN,NaN,NaN,NaN,553,2013-07-18 21:30:00,Ciudad Autónoma de Buenos Aires,Barrio Palermo,NaN,Por aspecto físico,NaN,En el acceso local,Controladores,Agresión verbal,NaN,Derivado,2013-07-24 12:45:04.487,Call Center,2013-07-19 12:48:28.177,2
3,060274,Florencio Varela,06,Buenos Aires,273,2012-03-11 04:30:00,Buenos Aires,Florencio Varela,Fcio. Varela,NaN,NaN,En las proximidades del local,Controladores,Agresión verbal - Agresión física,Se hizo denuncia policial,Derivado,2012-03-14 12:56:23.353,Call Center,2012-03-13 17:12:25.580,6
4,900014,Municipalidad de San Miguel De Tucuman,90,Tucumán,1209,2017-08-12 01:00:00,Tucumán,San Miguel de Tucumán,SAN MIGUEL DE TUCUMAN,Racial,El personal de control no esta identificado - ...,En el acceso local,Personas vinculadas a la empresa,NaN,NaN,Derivado,2017-08-14 16:47:16.497,Call Center,2017-08-14 09:27:25.947,90
5,NaN,NaN,NaN,NaN,1210,2017-07-28 03:00:00,Ciudad Autónoma de Buenos Aires,Barrio Palermo,NaN,Por aspecto físico - Por indumentaria,NaN,En el acceso local,Controladores - Personal de empresas de seguri...,Agresión verbal,NaN,Derivado,2017-08-15 13:30:25.280,Página web de RENCAP,2017-08-15 10:22:25.733,2
6,NaN,NaN,NaN,NaN,838,2015-05-24 05:00:00,Ciudad Autónoma de Buenos Aires,Barrio Palermo,NaN,NaN,NaN,En el interior del local - En las proximidades...,Controladores,Agresión física,NaN,Derivado,2015-06-12 14:00:56.993,Call Center,2015-06-10 13:32:48.400,2
7,060357,General Pueyrredón,06,Buenos Aires,38,2011-03-08 03:30:00,Buenos Aires,General Pueyrredón,Mar del Plata,NaN,NaN,En el interior del local,Personal de empresas de seguridad,Agresión física,Se solicitó asistencia policial,Derivado,2011-03-10 10:16:37.180,Call Center,2011-03-10 09:23:41.327,6
8,NaN,NaN,NaN,NaN,371,2012-07-18 00:30:00,Ciudad Autónoma de Buenos Aires,Barrio Saavedra,NaN,Por aspecto físico,Presencia de menores,En el acceso local,Controladores,NaN,NaN,Desestimado,2012-08-02 15:52:50.750,Call Center,2012-07-18 13:46:32.587,2
9,NaN,NaN,NaN,NaN,1143,2017-04-13 02:30:00,Ciudad Autónoma de Buenos Aires,Barrio Palermo,NaN,Por aspecto físico - Por indumentaria,No exhibe condiciones de adminsión y permanencia,En el acceso local,Controladores - Personal de empresas de seguridad,Agresión verbal,NaN,Desestimado,2017-04-20 16:44:10.387,Página web de RENCAP,2017-04-17 20:13:51.707,2


#### Centros de Acceso a la Justicia
Recurso:
- **Título:** Centros de Acceso a la Justicia (CAJ).
- **Descripción:** Centros de Acceso a la Justicia activos.
- **Fuente**: [datos.gob.ar](http://datos.gob.ar/dataset/justicia-centros-acceso-justicia--caj-)

In [25]:
df_justicia.head()

,oficina,fecha_apertura,provincia,localidad,domicilio,telefono,responsable,tipo_locacion,provincia_indec_id
0,CAJ - QUILMES,ago-14,Buenos Aires,"Bernal Oeste, Quilmes",Bermejo 1099 esq. Luis María Campos,sin teléfono,"Bassi, Emanuel",Espacio en CIC,6
1,CAJ - TRES DE FEBRERO,03/03/2017,Buenos Aires,Tres de Febrero,Murias 2160,Sin teléfono,"CACHAZA, Soledad",Local a la calle,6
2,CAJ - LOMAS DE ZAMORA,30/03/2017,Buenos Aires,Lomas de Zamora,Laprida 732,4243-8212,Erica Ruth Cowes,Local,6
3,CAJ - ONCE - EVA PERÓN,10/12/2014,CABA,Once,Avenida Rivadavia 2690,5300-4000 Interno 78490/78502,Temporalmente sin Coordinador designado. Depen...,Local,2
4,CAJ - TUCUMÁN Sur,04/11/2016,Tucumán,San Miguel de Tucumán,Avenida Jujuy 2253,Sin teléfono,"Sosa Lopez, Hugo Alfredo",casa,90


El DataFrame contiene datos de provincia, localidad y dirección. Se podría utilizar el recurso `/direcciones` de la API para intentar normalizar el campo `domicilio` del DataFrame, utilizando también el campo `provincia_indec_id`.

In [26]:
georef_normalize_entities(
    df_justicia,
    'direcciones',
    {'direccion': 'domicilio', 'provincia': 'provincia_indec_id'},
    {'campos': 'altura, id, nombre, nomenclatura'},
    prefix='gr_'
)

,gr_altura,gr_id,gr_nombre,gr_nomenclatura,oficina,fecha_apertura,provincia,localidad,domicilio,telefono,responsable,tipo_locacion,provincia_indec_id
0,NaN,NaN,NaN,NaN,CAJ - QUILMES,ago-14,Buenos Aires,"Bernal Oeste, Quilmes",Bermejo 1099 esq. Luis María Campos,sin teléfono,"Bassi, Emanuel",Espacio en CIC,6
1,2160.0,0684001002525,MURIAS 529,"MURIAS 529 2160, Tres de Febrero, Buenos Aires",CAJ - TRES DE FEBRERO,03/03/2017,Buenos Aires,Tres de Febrero,Murias 2160,Sin teléfono,"CACHAZA, Soledad",Local a la calle,6
2,732.0,0663804008105,LAPRIDA,"LAPRIDA 732, Pilar, Buenos Aires",CAJ - LOMAS DE ZAMORA,30/03/2017,Buenos Aires,Lomas de Zamora,Laprida 732,4243-8212,Erica Ruth Cowes,Local,6
3,2690.0,0206301001695,AV RIVADAVIA,"AV RIVADAVIA 2690, Comuna 9, Ciudad Autónoma d...",CAJ - ONCE - EVA PERÓN,10/12/2014,CABA,Once,Avenida Rivadavia 2690,5300-4000 Interno 78490/78502,Temporalmente sin Coordinador designado. Depen...,Local,2
4,NaN,NaN,NaN,NaN,CAJ - TUCUMÁN Sur,04/11/2016,Tucumán,San Miguel de Tucumán,Avenida Jujuy 2253,Sin teléfono,"Sosa Lopez, Hugo Alfredo",casa,90
5,NaN,NaN,NaN,NaN,CAJ - SANTIAGO DEL ESTERO,03/11/2011,Santiago del Estero,Santiago del Estero,"Santa Cruz 237, Barrio Alberdi",(03854) 283776,Maximo Apesteguia Uriburu,Local,86
6,461.0,5804201000775,MITRE,"MITRE 461, Chos Malal, Neuquén",CAJ - NEUQUÉN,01/05/2010,Neuquén,Neuquén,Mitre 461,(0299) 4488279,"RABERT SOTO, Sofia",Espacio en Municipalidad,58
7,968.0,0656801002800,NTRA SRA DEL BUEN VIAJE,"NTRA SRA DEL BUEN VIAJE 968, Morón, Buenos Aires",CAJ - MORON,27/09/2017,Buenos Aires,Moron,Ntra Sra de Buen Viaje 968,4489-7844,"DANERI CANESTRANI, Eduardo Santiago",Espacio en Municipalidad,6
8,NaN,NaN,NaN,NaN,CAJ - LA RIOJA,sep-14,La Rioja,La Rioja,Perón 451,(0380) 4439175,"OVELAR, VALERIA ELIZABETH",Casa,46
9,132.0,4202102002280,PELLEGRINI,"PELLEGRINI 132, Capital, La Pampa",CAJ - LA PAMPA,01/12/2012,La Pampa,Santa Rosa,Pellegrini 132,5300-4000 Interno 76129,Vanesa Ranocchia,Oficina,42


#### Geología - Yacimientos Minerales
Recurso:
- **Título:** Geología - Climatología - Yacimientos Minerales.
- **Descripción:** Ubicación de yacimientos minerales.
- **Fuente**: [datos.gob.ar](http://datos.gob.ar/dataset/energia-geologia---yacimientos-minerales)

In [27]:
df_minerales.head()

,yacimiento,provincia,mineraliza,geojson,geom
0,LA BLANCA,NEUQUEN,PLOMO PLATA ZINC,"{""type"":""Point"",""coordinates"":[-70.05444894523...",0101000020E610000057C46D177C8351C0FCDEBF18170F...
1,NaN,RIO NEGRO,ORO,"{""type"":""Point"",""coordinates"":[-71.55146232381...",0101000020E6100000F771A1284BE351C0E27C86739B0B...
2,EL COBRE,MENDOZA,COBRE,"{""type"":""Point"",""coordinates"":[-69.53502491979...",0101000020E6100000504529D93D6251C09072D7A919BB...
3,EL ORO,LA RIOJA,ORO,"{""type"":""Point"",""coordinates"":[-67.74985101207...",0101000020E6100000D66D198FFDEF50C076E02F90B430...
4,SAN ANTONIO,MENDOZA,COBRE,"{""type"":""Point"",""coordinates"":[-68.91892017385...",0101000020E610000086968F96CF3A51C04469998B6751...


El DataFrame contiene datos de coordenadas en la columna `geojson`, en formato GeoJSON. Para utilizar los datos, es necesario extraer la longitud y latitud y agregarlas en nuevas columnas. Para lograr esto, se puede utilizar la función `expand_geojson_column` mencionada anteriormente:

In [28]:
df_minerales_coords = expand_geojson_column(df_minerales)
df_minerales_coords

,longitud,latitud,yacimiento,provincia,mineraliza,geojson,geom
0,-70.054449,-38.117892,LA BLANCA,NEUQUEN,PLOMO PLATA ZINC,"{""type"":""Point"",""coordinates"":[-70.05444894523...",0101000020E610000057C46D177C8351C0FCDEBF18170F...
1,-71.551462,-42.090681,NaN,RIO NEGRO,ORO,"{""type"":""Point"",""coordinates"":[-71.55146232381...",0101000020E6100000F771A1284BE351C0E27C86739B0B...
2,-69.535025,-33.461721,EL COBRE,MENDOZA,COBRE,"{""type"":""Point"",""coordinates"":[-69.53502491979...",0101000020E6100000504529D93D6251C09072D7A919BB...
3,-67.749851,-29.190255,EL ORO,LA RIOJA,ORO,"{""type"":""Point"",""coordinates"":[-67.74985101207...",0101000020E6100000D66D198FFDEF50C076E02F90B430...
4,-68.918920,-32.635972,SAN ANTONIO,MENDOZA,COBRE,"{""type"":""Point"",""coordinates"":[-68.91892017385...",0101000020E610000086968F96CF3A51C04469998B6751...
5,-69.869752,-30.291752,DON CARLOS,SAN JUAN,PLOMO PLATA ZINC,"{""type"":""Point"",""coordinates"":[-69.86975190104...",0101000020E61000006CA8E003AA7751C04E2C2E40B04A...
6,-69.355298,-33.282552,AMELIA,MENDOZA,COBRE,"{""type"":""Point"",""coordinates"":[-69.35529752147...",0101000020E61000008CC4D031BD5651C00C2859AC2AA4...
7,-68.519605,-28.659445,VIRGEN DEL CARMEN,LA RIOJA,COBRE,"{""type"":""Point"",""coordinates"":[-68.51960524599...",0101000020E61000001D985C36412151C0A80CAE5DD1A8...
8,-69.441866,-36.653632,NIDIA;LUIS; JOSE; CARLOS,MENDOZA,COBRE,"{""type"":""Point"",""coordinates"":[-69.44186647324...",0101000020E6100000C0F2508A475C51C08999AC32AA53...
9,-66.173132,-22.505587,EL CONDOR,JUJUY,ORO,"{""type"":""Point"",""coordinates"":[-66.17313162014...",0101000020E6100000E29AA596148B50C04E59A7256E81...


Contando con las columnas `longitud` y `latitud`, se puede proceder a utilizar la función `georef_reverse_geocode` para realizar una geocodificación inversa de los datos, y obtener así la provincia, departamento y municipio en donde se encuentra cada punto.

In [29]:
georef_reverse_geocode(
    df_minerales_coords,
    {'lon': 'longitud', 'lat': 'latitud'},
    prefix='gr_'
)

,gr_departamento_id,gr_departamento_nombre,gr_municipio_id,gr_municipio_nombre,gr_provincia_id,gr_provincia_nombre,longitud,latitud,yacimiento,provincia,mineraliza,geojson,geom
0,58063,Loncopué,585063,Chorriaca,58,Neuquén,-70.054449,-38.117892,LA BLANCA,NEUQUEN,PLOMO PLATA ZINC,"{""type"":""Point"",""coordinates"":[-70.05444894523...",0101000020E610000057C46D177C8351C0FCDEBF18170F...
1,26014,Cushamen,260021,El Hoyo,26,Chubut,-71.551462,-42.090681,NaN,RIO NEGRO,ORO,"{""type"":""Point"",""coordinates"":[-71.55146232381...",0101000020E6100000F771A1284BE351C0E27C86739B0B...
2,50119,Tunuyán,500119,Tunuyán,50,Mendoza,-69.535025,-33.461721,EL COBRE,MENDOZA,COBRE,"{""type"":""Point"",""coordinates"":[-69.53502491979...",0101000020E6100000504529D93D6251C09072D7A919BB...
3,46042,Chilecito,460042,Chilecito,46,La Rioja,-67.749851,-29.190255,EL ORO,LA RIOJA,ORO,"{""type"":""Point"",""coordinates"":[-67.74985101207...",0101000020E6100000D66D198FFDEF50C076E02F90B430...
4,50049,Las Heras,500049,Las Heras,50,Mendoza,-68.918920,-32.635972,SAN ANTONIO,MENDOZA,COBRE,"{""type"":""Point"",""coordinates"":[-68.91892017385...",0101000020E610000086968F96CF3A51C04469998B6751...
5,70021,Calingasta,700021,Calingasta,70,San Juan,-69.869752,-30.291752,DON CARLOS,SAN JUAN,PLOMO PLATA ZINC,"{""type"":""Point"",""coordinates"":[-69.86975190104...",0101000020E61000006CA8E003AA7751C04E2C2E40B04A...
6,50126,Tupungato,500126,Tupungato,50,Mendoza,-69.355298,-33.282552,AMELIA,MENDOZA,COBRE,"{""type"":""Point"",""coordinates"":[-69.35529752147...",0101000020E61000008CC4D031BD5651C00C2859AC2AA4...
7,46098,Vinchina,460098,Vinchina,46,La Rioja,-68.519605,-28.659445,VIRGEN DEL CARMEN,LA RIOJA,COBRE,"{""type"":""Point"",""coordinates"":[-68.51960524599...",0101000020E61000001D985C36412151C0A80CAE5DD1A8...
8,50077,Malargüe,500077,Malargüe,50,Mendoza,-69.441866,-36.653632,NIDIA;LUIS; JOSE; CARLOS,MENDOZA,COBRE,"{""type"":""Point"",""coordinates"":[-69.44186647324...",0101000020E6100000C0F2508A475C51C08999AC32AA53...
9,38049,Rinconada,386077,Rinconada,38,Jujuy,-66.173132,-22.505587,EL CONDOR,JUJUY,ORO,"{""type"":""Point"",""coordinates"":[-66.17313162014...",0101000020E6100000E29AA596148B50C04E59A7256E81...


#### Puntos Digitales
Recurso:
- **Título:** Ubicación de los puntos digitales.
- **Descripción:** Listado de los puntos digitales y su ubicación geográfica.
- **Fuente**: [datos.gob.ar](http://datos.gob.ar/dataset/modernizacion-puntos-digitales)

In [30]:
df_puntos_digitales.head()

,id_pd,nombre_pd,direccion,nombre_institucional,id_provincia,id_departamento,id_localidad,id_municipio,provincia,departamento,localidad,municipio,mail_institucional,latitud,longitud,horarios,link_facebook,estado
0,292,Penal de Castañares,Houssay,Penal de Castañares,AR-A,1681,9508.0,SAL048,Salta,Capital,Salta,Salta,castanares@puntodigital.gob.ar,-24.730256,-65.406203,"Lunes 08:00 a 18:00 , Martes 08:00 a 18:00 , M...",NaN,Activo
1,270,Politécnico,Ayacucho 1667,Politecnico,AR-S,1684,11488.0,SFE264,Santa Fe,Rosario,Rosario,Rosario,politecnicorosario@puntodigital.gob.ar,-32.944243,-60.650539,"Lunes 08:00 a 20:00 , Martes 08:00 a 20:00 , M...",https://www.facebook.com/puntodigitalpolitecnico/,Activo
2,224,Eldorado,Misiones,CIC,AR-N,1542,8513.0,MIS031,Misiones,Eldorado,Eldorado,Eldorado,eldorado@puntodigital.gob.ar,-26.399318,-54.626341,Martes 16:00 a 20:00 - 07:00 a 13:00 - 13:30 a...,https://www.facebook.com/profile.php?id=100005...,Activo
3,394,9 de Julio - San Juan,Diagonal Sarmiento s/n,Hogar de día municipal,AR-J,1586,10418.0,SJU008,San Juan,Nueve de Julio,9 de Julio,Nueve de Julio,NaN,-31.669057,-68.3903618,NaN,NaN,Activo
4,113,Trevelin,Rio Futaleufu 1897,CIM centro integral de la mujer,AR-U,1371,5768.0,CHU043,Chubut,Futaleufu,Trevelin,Trevelin,trevelin@puntodigital.gob.ar,-43.101628,-71.469948,"Lunes 07:00 a 13:00 - 14:00 a 20:00 , Martes 0...",https://www.facebook.com/Punto-Digital-Treveli...,Activo


El DataFrame de Puntos Digitales es muy completo: cuenta con columnas para `direccion`, `localidad`, `municipio`, `departamento` y `provincia`. Sin embargo, los IDs utilizados para las entidades son ISO 3166-2, en vez de los utilizados por INDEC e IGN (por ejemplo, el ID utilizado para la provincia de Buenos Aires es "AR-B" en vez de "06"). Se podría utilizar nuevamente el recurso `/localidades` de la API para normalizar los datos y así agregar los IDs oficiales para las provincias, departamentos, municipios y localidades.

In [31]:
georef_normalize_entities(
    df_puntos_digitales,
    'localidades',
    {'nombre': 'localidad', 'provincia': 'provincia', 'municipio': 'municipio', 'departamento': 'departamento'},
    {'campos': 'id, nombre, municipio.id, departamento.id, provincia.id'},
    prefix='gr_'
)

,gr_departamento_id,gr_id,gr_municipio_id,gr_nombre,gr_provincia_id,id_pd,nombre_pd,direccion,nombre_institucional,id_provincia,...,provincia,departamento,localidad,municipio,mail_institucional,latitud,longitud,horarios,link_facebook,estado
0,66028,66028050003,660063,SALTA,66,292,Penal de Castañares,Houssay,Penal de Castañares,AR-A,...,Salta,Capital,Salta,Salta,castanares@puntodigital.gob.ar,-24.730256,-65.406203,"Lunes 08:00 a 18:00 , Martes 08:00 a 18:00 , M...",NaN,Activo
1,82084,82084270000,82210,ROSARIO,82,270,Politécnico,Ayacucho 1667,Politecnico,AR-S,...,Santa Fe,Rosario,Rosario,Rosario,politecnicorosario@puntodigital.gob.ar,-32.944243,-60.650539,"Lunes 08:00 a 20:00 , Martes 08:00 a 20:00 , M...",https://www.facebook.com/puntodigitalpolitecnico/,Activo
2,54042,54042020000,540154,ELDORADO,54,224,Eldorado,Misiones,CIC,AR-N,...,Misiones,Eldorado,Eldorado,Eldorado,eldorado@puntodigital.gob.ar,-26.399318,-54.626341,Martes 16:00 a 20:00 - 07:00 a 13:00 - 13:30 a...,https://www.facebook.com/profile.php?id=100005...,Activo
3,NaN,NaN,NaN,NaN,NaN,394,9 de Julio - San Juan,Diagonal Sarmiento s/n,Hogar de día municipal,AR-J,...,San Juan,Nueve de Julio,9 de Julio,Nueve de Julio,NaN,-31.669057,-68.3903618,NaN,NaN,Activo
4,26035,26035060000,260084,TREVELIN,26,113,Trevelin,Rio Futaleufu 1897,CIM centro integral de la mujer,AR-U,...,Chubut,Futaleufu,Trevelin,Trevelin,trevelin@puntodigital.gob.ar,-43.101628,-71.469948,"Lunes 07:00 a 13:00 - 14:00 a 20:00 , Martes 0...",https://www.facebook.com/Punto-Digital-Treveli...,Activo
5,58077,58077050000,585084,MANZANO AMARGO,58,233,Manzano Amargo,Ruta Provincia Nº 54 S/N,Comisión de Fomento Manzano Amargo,AR-Q,...,Neuquén,Minas,Manzano Amargo,Manzano Amargo,manzanoamargo@puntodigital.gob.ar,-36.754032,-70.763068,"Lunes 14:00 a 19:00 , Martes 14:00 a 19:00 , M...",NaN,Activo
6,NaN,NaN,NaN,NaN,NaN,451,Costa Sacate,Resistencia S/N,Nico Córdoba,AR-X,...,Córdoba,Rio Segundo,Costasacate,Costa Sacate,costasacate@puntodigital.gob.ar,-31.645981,-63.767766,NaN,NaN,Activo
7,06224,06224010000,060224,CHIVILCOY,06,44,Chivilcoy,Miguel Calderon 1271,Centro Integral Comunitario,AR-B,...,Buenos Aires,Chivilcoy,Chivilcoy,Chivilcoy,chivilcoy@puntodigital.gob.ar,-34.916846,-60.001681,"Lunes 10:00 a 16:00 , Martes 10:00 a 16:00 , M...",NaN,Activo
8,90084,90084010000,900014,SAN MIGUEL DE TUCUMAN,90,309,Casa de la ciudad,Catamarca 920,Casa de la Ciudad: Coordinación de Políticas S...,AR-T,...,Tucumán,Capital,San Miguel de Tucuman,San Miguel de Tucumán,tucumancasaciudad@puntodigital.gob.ar,-26.816862,-65.209454,"Lunes 08:30 a 13:00 - 16:30 a 21:30 , Martes 0...",https://www.facebook.com/PuntoDigitalCasaDeLaC...,Activo
9,NaN,NaN,NaN,NaN,NaN,605,Oberá / Misiones (provisorio),NaN,Oberá / Misiones (provisorio),AR-N,...,Misiones,Obera,NaN,Oberá,NaN,NaN,NaN,NaN,NaN,Activo


También podría interesarnos normalizar el campo `direccion` del dataframe, con el fin de obtener el nombre y ID de calle oficiales de cada dirección. Para lograr esto, se podría utilizar nuevamente el recurso `/direcciones` (partiendo del DataFrame original):

In [32]:
# Utilizar filas que tengan una dirección presente, y cuya altura no sea 0
df_puntos_digitales_dir = df_puntos_digitales.dropna(subset=['direccion'])
df_puntos_digitales_dir = df_puntos_digitales_dir[~df_puntos_digitales_dir['direccion'].str.endswith(' 0')]
df_puntos_digitales_dir = df_puntos_digitales_dir.reset_index(drop=True)

georef_normalize_entities(
    df_puntos_digitales_dir,
    'direcciones',
    {'direccion': 'direccion', 'provincia': 'provincia', 'departamento': 'departamento'},
    {'campos': 'id, nomenclatura'},
    prefix='gr_'
)

,gr_altura,gr_id,gr_nombre,gr_nomenclatura,id_pd,nombre_pd,direccion,nombre_institucional,id_provincia,id_departamento,...,provincia,departamento,localidad,municipio,mail_institucional,latitud,longitud,horarios,link_facebook,estado
0,NaN,6602805000600,AV BERNARDO HOUSSAY,"AV BERNARDO HOUSSAY, Capital, Salta",292,Penal de Castañares,Houssay,Penal de Castañares,AR-A,1681,...,Salta,Capital,Salta,Salta,castanares@puntodigital.gob.ar,-24.730256,-65.406203,"Lunes 08:00 a 18:00 , Martes 08:00 a 18:00 , M...",NaN,Activo
1,1667.0,8208427001215,AYACUCHO,"AYACUCHO 1667, Rosario, Santa Fe",270,Politécnico,Ayacucho 1667,Politecnico,AR-S,1684,...,Santa Fe,Rosario,Rosario,Rosario,politecnicorosario@puntodigital.gob.ar,-32.944243,-60.650539,"Lunes 08:00 a 20:00 , Martes 08:00 a 20:00 , M...",https://www.facebook.com/puntodigitalpolitecnico/,Activo
2,NaN,5404202002930,MISIONES,"MISIONES, Eldorado, Misiones",224,Eldorado,Misiones,CIC,AR-N,1542,...,Misiones,Eldorado,Eldorado,Eldorado,eldorado@puntodigital.gob.ar,-26.399318,-54.626341,Martes 16:00 a 20:00 - 07:00 a 13:00 - 13:30 a...,https://www.facebook.com/profile.php?id=100005...,Activo
3,NaN,NaN,NaN,NaN,394,9 de Julio - San Juan,Diagonal Sarmiento s/n,Hogar de día municipal,AR-J,1586,...,San Juan,Nueve de Julio,9 de Julio,Nueve de Julio,NaN,-31.669057,-68.3903618,NaN,NaN,Activo
4,NaN,NaN,NaN,NaN,113,Trevelin,Rio Futaleufu 1897,CIM centro integral de la mujer,AR-U,1371,...,Chubut,Futaleufu,Trevelin,Trevelin,trevelin@puntodigital.gob.ar,-43.101628,-71.469948,"Lunes 07:00 a 13:00 - 14:00 a 20:00 , Martes 0...",https://www.facebook.com/Punto-Digital-Treveli...,Activo
5,NaN,NaN,NaN,NaN,233,Manzano Amargo,Ruta Provincia Nº 54 S/N,Comisión de Fomento Manzano Amargo,AR-Q,1552,...,Neuquén,Minas,Manzano Amargo,Manzano Amargo,manzanoamargo@puntodigital.gob.ar,-36.754032,-70.763068,"Lunes 14:00 a 19:00 , Martes 14:00 a 19:00 , M...",NaN,Activo
6,NaN,NaN,NaN,NaN,451,Costa Sacate,Resistencia S/N,Nico Córdoba,AR-X,1398,...,Córdoba,Rio Segundo,Costasacate,Costa Sacate,costasacate@puntodigital.gob.ar,-31.645981,-63.767766,NaN,NaN,Activo
7,1271.0,0622401001310,MIGUEL CALDERON - C 61,"MIGUEL CALDERON - C 61 1271, Chivilcoy, Buenos...",44,Chivilcoy,Miguel Calderon 1271,Centro Integral Comunitario,AR-B,1221,...,Buenos Aires,Chivilcoy,Chivilcoy,Chivilcoy,chivilcoy@puntodigital.gob.ar,-34.916846,-60.001681,"Lunes 10:00 a 16:00 , Martes 10:00 a 16:00 , M...",NaN,Activo
8,NaN,NaN,NaN,NaN,309,Casa de la ciudad,Catamarca 920,Casa de la Ciudad: Coordinación de Políticas S...,AR-T,1659,...,Tucumán,Capital,San Miguel de Tucuman,San Miguel de Tucumán,tucumancasaciudad@puntodigital.gob.ar,-26.816862,-65.209454,"Lunes 08:30 a 13:00 - 16:30 a 21:30 , Martes 0...",https://www.facebook.com/PuntoDigitalCasaDeLaC...,Activo
9,NaN,6602805011915,MARTIN FIERRO,"MARTIN FIERRO, Capital, Salta",290,Santa Cecilia,Martín Fierro,CIC Santa Cecilia,AR-A,1681,...,Salta,Capital,Salta,Salta,santacecilia@puntodigital.gob.ar,-24.830325,-65.397813,"Lunes 09:00 a 12:00 - 15:00 a 22:00 , Martes 0...",NaN,Activo


#### ARSAT - Puntos de Conexión 
Recurso:
- **Título:** Puntos de conexión al Plan Federal de Internet.
- **Descripción:** La tabla muestra la ubicación precisa de los puntos de acceso al Plan Federal de Internet que se encuentran en servicio junto a otros datos como la población de la localidad y la fecha de conexión.
- **Fuente**: [datos.gob.ar](http://datos.gob.ar/dataset/arsat-puntos-conexion-al-plan-federal-internet)

In [33]:
df_arsat_internet.head()

,ID Sitio,Traza,Sub-Traza,Localidad,Departamento,Provincia,Poblacion Censo 2010,Lat_sitio,Lon_sitio,Fecha Real de Conexión,Estado
0,315,TOSTADO - RAFAELA,SUNCHALES - RAFAELA,Ataliva,Castellanos,Santa Fe,1680.0,-31.000564,-61.432581,03/08/18,Conectado
1,1354,CONCEPCION - ROSARIO,CONCEPCION DEL URUGUAY - GUALEGUAYCHU,Faustino M. Parera,Gualeguaychú,Entre Rios,158.0,-32.803700,-58.881475,29/06/18,Conectado
2,2580,FORMOSA - PICHANAL,CORONEL JUAN SOLA - HICKMAN,Pluma de Pato,Rivadavia,Salta,529.0,-23.372183,-63.097878,23/02/18,Conectado
3,3007,MERCEDES - CHASCOMUS,LOBOS - SAN MIGUEL DEL MONTE,San Miguel del Monte,Monte,Buenos Aires,17005.0,-35.447556,-58.814750,31/07/17,Conectado
4,1045,JUNIN - LUJAN,LUJAN - CARMEN DE ARECO,Cucullu,San Andrés de Giles,Buenos Aires,551.0,-34.445308,-59.360469,09/11/17,Conectado


El DataFrame cuenta con columnas para nombres de provincias, departamentos y localidades, pero no para los ID de las mismas. Se podría utilizar una vez más el recurso `/localidades` para agregar los ID de las tres entidades al DataFrame. Haciendo esto, también se agregaría (en algunos casos) el municipio relacionado a cada localidad.

In [34]:
georef_normalize_entities(
    df_arsat_internet,
    'localidades',
    {'nombre': 'Localidad', 'provincia': 'Provincia', 'departamento': 'Departamento'},
    {'campos': 'id, nombre, departamento.id, provincia.id, municipio.id, municipio.nombre'},
    prefix='gr_'
)

,gr_departamento_id,gr_id,gr_municipio_id,gr_municipio_nombre,gr_nombre,gr_provincia_id,ID Sitio,Traza,Sub-Traza,Localidad,Departamento,Provincia,Poblacion Censo 2010,Lat_sitio,Lon_sitio,Fecha Real de Conexión,Estado
0,82021,82021030000,822119,Ataliva,ATALIVA,82,315,TOSTADO - RAFAELA,SUNCHALES - RAFAELA,Ataliva,Castellanos,Santa Fe,1680.0,-31.000564,-61.432581,03/08/18,Conectado
1,30056,30056040000,None,None,FAUSTINO M. PARERA,30,1354,CONCEPCION - ROSARIO,CONCEPCION DEL URUGUAY - GUALEGUAYCHU,Faustino M. Parera,Gualeguaychú,Entre Rios,158.0,-32.803700,-58.881475,29/06/18,Conectado
2,66133,66133060000,660329,Rivadavia Banda Norte,PLUMA DE PATO,66,2580,FORMOSA - PICHANAL,CORONEL JUAN SOLA - HICKMAN,Pluma de Pato,Rivadavia,Salta,529.0,-23.372183,-63.097878,23/02/18,Conectado
3,06547,06547020000,060547,Monte,SAN MIGUEL DEL MONTE,06,3007,MERCEDES - CHASCOMUS,LOBOS - SAN MIGUEL DEL MONTE,San Miguel del Monte,Monte,Buenos Aires,17005.0,-35.447556,-58.814750,31/07/17,Conectado
4,06728,06728020000,060728,San Andrés de Giles,CULULLU,06,1045,JUNIN - LUJAN,LUJAN - CARMEN DE ARECO,Cucullu,San Andrés de Giles,Buenos Aires,551.0,-34.445308,-59.360469,09/11/17,Conectado
5,14161,14161030000,141435,Corralito,CORRALITO,14,1010,CORDOBA - RIO CUARTO,CORDOBA - CORRALITO,Corralito,Tercero Arriba,Cordoba,1727.0,-32.026750,-64.196378,06/01/16,Conectado
6,62042,62042570000,620098,Campo Grande,VILLA SAN ISIDRO,62,3557,SANTA ISABEL - NEUQUEN,BARDA DEL MEDIO - SARGENTO VIDAL,Villa San Isidro,General Roca,Rio Negro,607.0,-38.706303,-68.171708,26/10/17,Conectado
7,38049,38049020000,386070,Mina Pirquitas,LIVIARA,38,2031,LA QUIACA - SANTA MARIA,LA QUIACA - SAN ANTONIO DE LOS COBRES,Liviara,Rinconada,Jujuy,46.0,-22.519722,-66.337778,21/09/18,Conectado
8,NaN,NaN,NaN,NaN,NaN,NaN,627,ROSARIO - CORDOBA,CAÑADA de GOMEZ - MARCOS JUAREZ,Cañada de Gomez EDT,Iriondo,Santa Fe,29205.0,-32.816306,-61.412528,07/07/15,Conectado
9,54056,54056030000,540203,San Vicente,SAN VICENTE,54,3038,SAN JOSE - PUERTO IGUAZU,ARISTOBULO DEL VALLE - SAN VICENTE,San Vicente,Guaraní,Misiones,21068.0,-26.993983,-54.487300,21/12/17,Conectado


Ya que el DataFrame cuenta con datos de posición (`Lat_sitio` y `Lon_sitio`), se podría intentar también utilizar el recurso `/ubicacion` para normalizar los datos:

In [35]:
georef_reverse_geocode(
    df_arsat_internet,
    {'lon': 'Lon_sitio', 'lat': 'Lat_sitio'},
    prefix='gr_'
)

,gr_departamento_id,gr_departamento_nombre,gr_municipio_id,gr_municipio_nombre,gr_provincia_id,gr_provincia_nombre,ID Sitio,Traza,Sub-Traza,Localidad,Departamento,Provincia,Poblacion Censo 2010,Lat_sitio,Lon_sitio,Fecha Real de Conexión,Estado
0,82021,Castellanos,822119,Ataliva,82,Santa Fe,315,TOSTADO - RAFAELA,SUNCHALES - RAFAELA,Ataliva,Castellanos,Santa Fe,1680.0,-31.000564,-61.432581,03/08/18,Conectado
1,30056,Gualeguaychú,None,None,30,Entre Ríos,1354,CONCEPCION - ROSARIO,CONCEPCION DEL URUGUAY - GUALEGUAYCHU,Faustino M. Parera,Gualeguaychú,Entre Rios,158.0,-32.803700,-58.881475,29/06/18,Conectado
2,66133,Rivadavia,660329,Rivadavia Banda Norte,66,Salta,2580,FORMOSA - PICHANAL,CORONEL JUAN SOLA - HICKMAN,Pluma de Pato,Rivadavia,Salta,529.0,-23.372183,-63.097878,23/02/18,Conectado
3,06547,Monte,060547,Monte,06,Buenos Aires,3007,MERCEDES - CHASCOMUS,LOBOS - SAN MIGUEL DEL MONTE,San Miguel del Monte,Monte,Buenos Aires,17005.0,-35.447556,-58.814750,31/07/17,Conectado
4,06728,San Andrés de Giles,060728,San Andrés de Giles,06,Buenos Aires,1045,JUNIN - LUJAN,LUJAN - CARMEN DE ARECO,Cucullu,San Andrés de Giles,Buenos Aires,551.0,-34.445308,-59.360469,09/11/17,Conectado
5,14161,Tercero Arriba,141435,Corralito,14,Córdoba,1010,CORDOBA - RIO CUARTO,CORDOBA - CORRALITO,Corralito,Tercero Arriba,Cordoba,1727.0,-32.026750,-64.196378,06/01/16,Conectado
6,62042,General Roca,620140,Contralmirante Cordero,62,Río Negro,3557,SANTA ISABEL - NEUQUEN,BARDA DEL MEDIO - SARGENTO VIDAL,Villa San Isidro,General Roca,Rio Negro,607.0,-38.706303,-68.171708,26/10/17,Conectado
7,38049,Rinconada,386070,Mina Pirquitas,38,Jujuy,2031,LA QUIACA - SANTA MARIA,LA QUIACA - SAN ANTONIO DE LOS COBRES,Liviara,Rinconada,Jujuy,46.0,-22.519722,-66.337778,21/09/18,Conectado
8,82056,Iriondo,820126,Cañada de Gómez,82,Santa Fe,627,ROSARIO - CORDOBA,CAÑADA de GOMEZ - MARCOS JUAREZ,Cañada de Gomez EDT,Iriondo,Santa Fe,29205.0,-32.816306,-61.412528,07/07/15,Conectado
9,54056,Guaraní,540203,San Vicente,54,Misiones,3038,SAN JOSE - PUERTO IGUAZU,ARISTOBULO DEL VALLE - SAN VICENTE,San Vicente,Guaraní,Misiones,21068.0,-26.993983,-54.487300,21/12/17,Conectado


## Ideas y Mejoras
- Las funciones `georef_normalize_entities` y `georef_reverse_geocode` podrían ser mejoradas para poder manejar DataFrames de más de 5000 filas.
- Modificar los parámetros `length` y `random_state` en los llamados a `load_csv` para procesar distintos conjuntos de filas de cada DataFrame.
- Agregar manejo de errores a las funciones `georef_normalize_entities` y `georef_reverse_geocode`.

## Más información
- [Repositorio `georef-ar-api`](https://github.com/datosgobar/georef-ar-api)
- [Repositorio `georef-ar-etl`](https://github.com/datosgobar/georef-ar-etl)
- [Dataset de Georef API en datos.gob.ar](http://datos.gob.ar/dataset/modernizacion-servicio-normalizacion-datos-geograficos)
- [APIs del equipo de Datos de la Nación Argentina](http://datos.gob.ar/apis)
- [Portal de datos abiertos de la Nación Argentina](http://datos.gob.ar/)